<a href="https://colab.research.google.com/github/TeaWithLucas/ML-text-categorisation/blob/main/ml_text_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-- Author: C1600431 --

Part 2 Q1


Instructions: 
 

1.   Upload the datasets_coursework1.zip to the base file directory.
2.   (Optional) Change any of the Setting as you desire in the setting heading
3.   Run all the code using Runtime > Run all (Ctrl+F9) or run all the code sequentially manually
4.   View the execution section to see the excution process and outputs and the subheading Evaluation for the final results
5.   (Optional) Adjust any of the Settings as you desire
6.   (Optional) Run the Settings and Execution section and review the results

# Imports

In [41]:
import numpy as np
#import requests
#import pandas as pd
import tqdm
import matplotlib.pyplot as plt

#File Management
import os
import zipfile

#MLP
import sklearn
import sklearn.metrics as sklmet
import sklearn.model_selection as sklms
import sklearn.feature_extraction as sklfe
import sklearn.feature_selection as sklfs
#import spacy
!pip install zeugma
from  zeugma import embeddings
import nltk
nltk.download('stopwords') # If needed
nltk.download('punkt') # If needed
nltk.download('wordnet') # If needed
nltk.download('averaged_perceptron_tagger') # If needed

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Settings
These can change the functionality of the program easily

In [42]:
# set the zip containing the datasets
zipped_data="datasets_coursework1.zip"
# directory of the category folders
data_path_folder = "/content/bbc/"

# Choose the training, test and dev split (training split is the remaining amount)
test_size=20 #percent
dev_size=20 #percent

# Choose most frequent counts for these feature sets
token_max = 2000
tagged_max = 2000
ngram_max = 2000

# Choose the feature sets you want enabled
#feature_sets_enabled=['gen_basic']
#feature_sets_enabled=['bigram','gen','adjverb','embedding_glove']
feature_sets_enabled=['word_tagged','adjverb','embedding_glove']

# word_tagged tags
word_tags = ['EX','FW','JJ','LS','MD','NN','PDT','RB','RP','SYM','UH','VB','WDT','WP','WRB'] #ignoring puncuation, 'CD': Cardinal number, 'CC':Coordinating conjunction,'POS':Possessive ending, 'IN':Preposition or subordinating conjunction, 'TO':to, 'DT':Determiner,'PRP': 	Personal pronoun 

# Choose the feature selection scoring model
#scoring_model=sklfs.mutual_info_classif
scoring_model=sklfs.f_classif

# models and number of features for the development set refinement
model_choices = ((sklearn.svm.SVC,{'kernel':'linear'}),
                (sklearn.svm.LinearSVC,{'max_iter':10000}),
                (sklearn.svm.SVC,{'kernel':'rbf', 'gamma':0.7}),
                (sklearn.svm.SVC,{'kernel':'poly', 'degree':3, 'gamma':'scale'}))

list_num_features=[10,100,250,500,750,1000,1500,2000,3000]

# Functions

## Data processing

In [43]:
#for unzipping a file
def unzip_data(zip_file):
  with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()

#for a folder, use all the subfolders as catgories and their files as samples
def get_data(data_path):
  categories = {}
  num_files = 0
  print("Loading from folder:", data_path)
  for subdir, dirs, files in os.walk(data_path):
    cat_folders = {}
    for dir in dirs: #for each subdirectory, add a category
      categories[dir] = []
    for file in tqdm.tqdm_notebook(files): #for each file in directory
      fldr_name = os.path.basename(subdir)
      if fldr_name in categories: #if in a subfolder
        num_files += 1
        file_path = os.path.join(subdir, file)
        text = open(file_path, "r",errors='ignore').read()
        categories[fldr_name].append(text) #add text content to the relevent category
  num_samples = 0
  for cat in categories:
    num_samples += len(categories[cat])
    print("Category", cat, "Num Samples:", len(categories[cat]))
  print("Total Num Files:", num_files, ", Num Samples:", num_samples)
  return categories

#fallen all the samples into two lists, one for the samples, and the other for their category
def format_data_categories(categories):
  x = []
  y = []
  for cat in categories:
    for data in categories[cat]:
      x.append(data)
      y.append(cat)
  print(len(x),x[:5])
  print(len(y),y[:5])
  return x,y

## Data Splitting

In [44]:
#split the data according to the settings and return the different sets
def train_test_dev_split(x,y):
  #turning percentages into splits
  first_split = (test_size + dev_size)/100
  second_split = (dev_size/100)/first_split
  print("first split", first_split, "second split", second_split)

  #first split gets the training set
  x_train, x_temp, y_train, y_temp = sklms.train_test_split(X_full, Y_full, test_size=first_split)
  #second split gets the development and testing sets
  x_test, x_dev, y_test, y_dev = sklms.train_test_split(x_temp, y_temp, test_size=second_split)

  data_total_size = len(x_train)+len(x_dev)+len(x_test)
  print ("Total size: ", data_total_size)
  print ("\nTRAINING SET\n")
  print ("Size training set: "+str(len(x_train))+", " + str(int(len(x_train)/data_total_size*100)) + "%")
  for i in range(5):
    print ("cat:", y_train[i])
    print (x_train[i][:200].replace("\n"," ")+"...")
  print ("\nTEST SET\n")
  print ("Size test set: "+str(len(x_test))+", " + str(int(len(x_test)/data_total_size*100)) + "%")
  for i in range(5):
    print ("cat:", y_test[i])
    print (x_test[i][:200].replace("\n"," ")+"...")
  print ("\nDEV SET\n")
  print ("Size development set: "+str(len(x_dev))+", " + str(int(len(x_dev)/data_total_size*100)) + "%")
  for i in range(5):
    print ("cat:", y_dev[i])
    print (x_dev[i][:200].replace("\n"," ")+"...")
  
  return x_train, y_train, x_test, y_test, x_dev, y_dev

## Feature Extraction

In [45]:
#take the raw string samples and split them into lists of words
def tokenise_data(raw_data):
  tokenised = []
  for sample in tqdm.notebook.tqdm(raw_data):
    tok=nltk.tokenize.word_tokenize(sample)
    tokenised.append(tok)
  return tokenised

#take the lists of words and add a tag them with thier part-of-speech tag
def tag_data(tokenised_data):
  tagged=[]
  for sample in tqdm.notebook.tqdm(tokenised_data):
    pos=nltk.pos_tag(sample) 
    tagged.append(pos)
  return tagged

#take the lists of tagged words, filter them with allowed tags, 
# then append the tag and return a list of flattend samples
def flatten_tag_data(tagged_data, allowed_tags=[]):
  flattened_data=[]
  for sample in  tagged_data:
    filtered_sample = []
    for item in sample:
      tag = item[1]
      token = item[0] + "/" + tag
      if len(allowed_tags) < 1 or any(x in tag for x in allowed_tags):
        filtered_sample.append(token)
    flattened_data.append(filtered_sample)
  return flattened_data

# use the lists of words to generate a volcabulary of the most frequent word counts
def token_vocabulary(token_data,max_features=5000):
  tfidf_vectorizer = sklfe.text.TfidfVectorizer(input='content', lowercase=False, preprocessor=lambda x: x, tokenizer=lambda x: x,max_features=max_features)
  tfidf_vectorizer.fit_transform(token_data)
  print("Token Vocabulary Length: ", len(tfidf_vectorizer.vocabulary_))
  print(list(tfidf_vectorizer.vocabulary_.keys()))
  return tfidf_vectorizer.vocabulary_, 'token'

# use the lists of tagged words, filter and flatten them
# to generate a volcabulary of the most frequent word counts
def tagged_vocabulary(tagged_data,allowed_tags,max_features=5000):
  flattened_data = flatten_tag_data(tagged_data,allowed_tags)
  tfidf_vectorizer = sklfe.text.TfidfVectorizer(input='content', lowercase=False, preprocessor=lambda x: x, tokenizer=lambda x: x,max_features=max_features)
  tfidf_vectorizer.fit_transform(flattened_data)
  print("Tagged Vocabulary Length: ", len(tfidf_vectorizer.vocabulary_))
  print(list(tfidf_vectorizer.vocabulary_.keys()))
  return tfidf_vectorizer.vocabulary_, 'tagged'

# use the raw sample strings to generate a volcabulary of the most frequent ngrams counts of a set ngrams range 
def ngrams_vocabulary(training_set,ngram_range,max_features=5000):
  tfidf_vectorizer = sklfe.text.TfidfVectorizer(ngram_range=ngram_range, stop_words='english', norm=None, smooth_idf=False, use_idf=False,max_features=max_features)
  tfidf_vectorizer.fit_transform(training_set)
  print("n-grams Vocabulary Length: ", )
  print(list(tfidf_vectorizer.vocabulary_.keys()))
  return tfidf_vectorizer.vocabulary_, 'raw'

## Data Vectorisation

In [46]:
# vectorise a set of raw sample strings given a pregenerated volcabulary
def tfidf_vectorise(training_set,Vocabulary):
  tfidf_vectorizer = sklfe.text.TfidfVectorizer(vocabulary=Vocabulary)
  return tfidf_vectorizer.fit_transform(training_set)

# vectorise a set of lists of words given a pregenerated volcabulary
def tokened_vectorise(Tokenized_data,Vocabulary):
  tfidf_vectorizer = sklfe.text.TfidfVectorizer(input='content', lowercase=False, preprocessor=lambda x: x, tokenizer=lambda x: x,vocabulary=Vocabulary)
  return tfidf_vectorizer.fit_transform(Tokenized_data)

# vectorise a set of raw sample strings given a pretrained embedding
def embedding_vectorise(training_set,embedding):
  embedding_trans = embeddings.EmbeddingTransformer(embedding)
  return embedding_trans.transform(training_set)

# vectorise all the different types of data generated using the feature sets chosen
# in the settings, concatenate them together and return them
def vectorise_data(raw_data,token_data,tagged_data,vocabs):
  flattened_tag_data = flatten_tag_data(tagged_data)
  vectorised = np.empty((len(raw_data),0))
  for name in vocabs:
    if name in feature_sets_enabled:
      vocab = vocabs[name][0]
      vector = np.empty((len(raw_data),0))
      if vocabs[name][1] == 'raw':
        vector = tfidf_vectorise(raw_data,vocab).toarray()
      elif vocabs[name][1] == 'tagged':
        vector = tokened_vectorise(flattened_tag_data,vocab).toarray()
      elif vocabs[name][1] == 'token':
        vector = tokened_vectorise(token_data,vocab).toarray()
      else:
        print("problem with vocabulary", name)
      print(name + " Num Features:", vector.shape)
      vectorised = np.concatenate((vectorised, vector), axis=1)
  embeddings = [i[10:] for i in feature_sets_enabled if i.startswith('embedding_')]
  for embedding in embeddings:
    vector = embedding_vectorise(raw_data, embedding)
    print(embedding + " Num Features:", vector.shape)
    vectorised = np.concatenate((vectorised, vector), axis=1)
  print("Total Num Features:", vectorised.shape)
  return vectorised



## Feature selection

In [47]:
# create a feature filter class using the best features ordered 
# the scoring model set in the settings and data sets given
# and limited the number of features given by num
def select_KBest(x, y, num):
  sentanalysis = sklfs.SelectKBest(scoring_model, k=num).fit(x,y)
  return sentanalysis

# create a feature filter class using the best features ordered 
# the scoring model set in the settings and data sets given
# and limited the percentage of features given by percentile
def select_percentile(x, y, percentile):
  sentanalysis = sklfs.SelectPercentile(scoring_model, percentile=percentile).fit(x,y)
  return sentanalysis

# use a feature filter class to filter features from a training set
def reduce_sentanalysis(sentanalysis, x):
  x_reduced = sentanalysis.transform(x)
  return x_reduced

## Classifier

In [48]:
# train a model based upon the training sets given,
# filtering using the feature filter class,
# and either using a default model or a model class and parameters given
# returning the classifier model generated 
def train_svm_classifier(x, y, sentanalysis,clf=None):
  x_reduced = reduce_sentanalysis(sentanalysis,x)
  if clf==None:
    svm_clf=sklearn.svm.SVC(kernel="linear",gamma='scale')
  else:
    svm_clf=clf[0](**clf[1])
  svm_clf.fit(x_reduced,y) #Train the new SVM model. This may take a while.
  #print(svm_clf)
  return svm_clf

# use a classifier model to predict the values of a given set of test data
# and a feature filter class 
def predict_svm_classifier(svm_clf, x, sentanalysis):
  x_reduced = reduce_sentanalysis(sentanalysis,x)
  return svm_clf.predict(x_reduced) 

# use a classifier model to score a given set of test data, its gold classifers
# and a feature filter class 
def score_svm_classifier(svm_clf, x, y, sentanalysis):
  x_reduced = reduce_sentanalysis(sentanalysis,x)
  return svm_clf.score(x_reduced,y) 

# use a classifier model to plot a confusion matrix 
# for a given set of test data, its gold classifers
# and a feature filter class 
def plot_confusion_matrix(svm_clf, x, y, sentanalysis, title=""):
  x_reduced = reduce_sentanalysis(sentanalysis,x)
  disp = sklmet.plot_confusion_matrix(svm_clf,x_reduced,y,values_format='d',xticks_rotation='vertical',normalize=None)
  if title != "":
    disp.ax_.set_title(title)
  return 

## Development Set Tuning

In [49]:
# using a set of model classes, their parameters, feature counts, 
# training sets, developement sets and thier gold classifers
# for each model and feature count, filter the top scoring features, 
# train a classifer with those features, score that combination,
# find the overall top scoring model, feature count combination
# then return the best score, best model classes/parameters, 
# best generated model and best feature filter
def best_model(models, feature_counts, x, y, xd, yd):
  best_score=0.0
  for clf in models:
    best_score_features=0.0
    best_svm_clf_features=None
    print ("Testing clf:\n", clf)
    for num_features in feature_counts:
      
      sent_analysis_dev = select_KBest(x, y, num_features)
      svm_clf_dev=train_svm_classifier(x, y, sent_analysis_dev, clf=clf)

      score = score_svm_classifier(svm_clf_dev, xd, yd, sent_analysis_dev)
      print ("Score with "+str(num_features)+"\tfeatures: "+str(round(score,3)))
      if score>best_score_features:
        best_score_features=score
        best_num_features_clf=num_features
        best_svm_clf_features=svm_clf_dev
        best_sentanalysis_clf=sent_analysis_dev
    print ("\nBest score for this clf is "+str(round(best_score_features,3))+" with "+str(best_num_features_clf)+" features.\n")
    if best_score_features>best_score: 
      best_score=best_score_features
      best_num_features=best_num_features_clf
      best_clf=clf
      best_svm_clf=best_svm_clf_features
      best_sentanalysis=best_sentanalysis_clf
  print ("\nBest score overall in the dev set is "+str(round(best_score,3))+" with "+str(best_num_features)+" features using clf:")
  print (best_svm_clf)
  
  return best_score, best_clf, best_svm_clf, best_sentanalysis

# Execution

## Data collection

Unzip Data

In [50]:
unzip_data(zipped_data)

Get Categories and Data from files

In [51]:
data_categories = get_data(data_path_folder)

Loading from folder: /content/bbc/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app



Category politics Num Samples: 417
Category sport Num Samples: 511
Category tech Num Samples: 401
Category business Num Samples: 510
Category entertainment Num Samples: 386
Total Num Files: 2225 , Num Samples: 2225


Check data retrived is sane

In [52]:
  for cat in data_categories:
    print(cat,data_categories[cat][:5])

politics ['Guantanamo pair\'s passport ban\n\nThe government has written to two of the British men freed from Guantanamo Bay telling them they will not be allowed passports.\n\nA letter sent to Martin Mubanga said his British passport would not be issued in the light of evidence gathered against him by the US. This suggested he was likely to take part in action against UK or allied targets if he left Britain, it said. An identical letter has been sent to Feroz Abbasi, the men\'s solicitor says.\n\nIt is not known whether the other two men released from the Cuba detention camp in January - Richard Belmar and Moazzam Begg - have also received letters. The government is implementing the rarely used Royal Prerogative in order to withdraw the men\'s passports. It is only the 13th time the power has been used since 1947 - the last time was in 1976. The letter, from the Home Office, says: "I am writing to inform you that on the basis of the information which has come to light during your dete

## Model Selection

Combine into full dataset

In [53]:
X_full, Y_full = format_data_categories(data_categories)

2225 ['Guantanamo pair\'s passport ban\n\nThe government has written to two of the British men freed from Guantanamo Bay telling them they will not be allowed passports.\n\nA letter sent to Martin Mubanga said his British passport would not be issued in the light of evidence gathered against him by the US. This suggested he was likely to take part in action against UK or allied targets if he left Britain, it said. An identical letter has been sent to Feroz Abbasi, the men\'s solicitor says.\n\nIt is not known whether the other two men released from the Cuba detention camp in January - Richard Belmar and Moazzam Begg - have also received letters. The government is implementing the rarely used Royal Prerogative in order to withdraw the men\'s passports. It is only the 13th time the power has been used since 1947 - the last time was in 1976. The letter, from the Home Office, says: "I am writing to inform you that on the basis of the information which has come to light during your detentio

Split into training, testing and dev


In [54]:
X_train_data, Y_train, X_test_data, Y_test, X_dev_data, Y_dev = train_test_dev_split(X_full, Y_full)

first split 0.4 second split 0.5
Total size:  2225

TRAINING SET

Size training set: 1335, 60%
cat: business
Fresh hope after Argentine crisis  Three years after Argentina was hit by a deadly economic crisis, there is fresh hope.  The country's economy is set to grow about 8% this year after seeing 9% growth...
cat: business
Yukos heading back to US courts  Russian oil and gas company Yukos is due in a US court on Thursday as it continues to fight for its survival.  The firm is in the process of being broken up by Russian...
cat: tech
Blind student 'hears in colour'  A blind student has developed software that turns colours into musical notes so that he can read weather maps.  Victor Wong, a graduate student from Hong Kong studying...
cat: business
Japanese growth grinds to a halt  Growth in Japan evaporated in the three months to September, sparking renewed concern about an economy not long out of a decade-long trough.  Output in the period gre...
cat: sport
Isinbayeva heads for Birmi

## Feature engineering

Tokenise the data using NLTK word tokenizer 

In [55]:
X_train_token_data = tokenise_data(X_train_data)
X_test_token_data = tokenise_data(X_test_data)
X_dev_token_data = tokenise_data(X_dev_data)

Tag the datasets using NLTK part-of-speech tagging 

In [56]:
X_train_tag_data = tag_data(X_train_token_data)
X_test_tag_data = tag_data(X_test_token_data)
X_dev_tag_data = tag_data(X_dev_token_data)

Get volcabularies for count vectorisations

In [57]:
vocabularies={}
#
vocabularies['gen_basic'] = token_vocabulary(X_train_token_data,token_max)

vocabularies['bigram'] = ngrams_vocabulary(X_train_data,(2,2),ngram_max)
vocabularies['trigram'] = ngrams_vocabulary(X_train_data,(3,3),ngram_max)

vocabularies['word_tagged'] = tagged_vocabulary(X_train_tag_data,word_tags,tagged_max)
vocabularies['verb'] = tagged_vocabulary(X_train_tag_data,['VB'],tagged_max)
vocabularies['adj'] = tagged_vocabulary(X_train_tag_data,['JJ'],tagged_max)
vocabularies['adjverb'] = tagged_vocabulary(X_train_tag_data,['VB','JJ'],tagged_max)

Token Vocabulary Length:  2000
['hope', 'after', 'crisis', 'Three', 'years', 'Argentina', 'was', 'hit', 'by', 'a', 'economic', ',', 'there', 'is', '.', 'The', 'country', "'s", 'economy', 'set', 'to', 'grow', 'about', '8', '%', 'this', 'year', 'seeing', '9', 'growth', 'last', 'from', '2002', 'when', 'output', 'fell', '11', 'unemployment', 'rate', 'improving', 'too', ':', 'It', 'below', '13', 'the', 'end', 'of', 'down', '20', 'in', 'May', 'problems', 'remain', 'but', 'overall', 'picture', 'one', 'Even', 'International', '(', ')', '``', 'authorities', 'are', 'should', 'be', 'strong', 'performance', "''", 'Thomas', 'an', 'director', 'said', 'earlier', 'month', 'has', 'made', 'recovery', 'and', 'which', '2001', 'government', 'debt', 'its', 'private', 'at', 'least', 'decisions', 'were', 'bank', 'accounts', 'as', 'part', 'measures', 'introduced', 'deal', 'with', 'massive', 'In', 'response', 'took', 'where', 'lives', 'lost', 'police', 'Two', 'three', 'finance', 'minister', 'less', 'than', 'on'

Vectorise the data only using the vocabularies and embeddings in the 'feature_sets_enabled' variable in the settings

In [58]:
X_train = vectorise_data(X_train_data,X_train_token_data,X_train_tag_data,vocabularies)
X_test = vectorise_data(X_test_data,X_test_token_data,X_test_tag_data,vocabularies)
X_dev = vectorise_data(X_dev_data,X_dev_token_data,X_dev_tag_data,vocabularies)

word_tagged Num Features: (1335, 2000)
adjverb Num Features: (1335, 2000)
glove Num Features: (1335, 25)
Total Num Features: (1335, 4025)
word_tagged Num Features: (445, 2000)
adjverb Num Features: (445, 2000)
glove Num Features: (445, 25)
Total Num Features: (445, 4025)
word_tagged Num Features: (445, 2000)
adjverb Num Features: (445, 2000)
glove Num Features: (445, 25)
Total Num Features: (445, 4025)


## Feature Refinement and Selection

Using the training sample, select the best features then score it based upon the development.

Work through the models and number of feature provided to find the best combination

In [59]:
best_score, best_clf, best_svm_clf, best_sent_analysis = best_model(model_choices,list_num_features, X_train, Y_train, X_dev, Y_dev)

Testing clf:
 (<class 'sklearn.svm._classes.SVC'>, {'kernel': 'linear'})
Score with 10	features: 0.82
Score with 100	features: 0.935
Score with 250	features: 0.957
Score with 500	features: 0.96
Score with 750	features: 0.953
Score with 1000	features: 0.957
Score with 1500	features: 0.955
Score with 2000	features: 0.966
Score with 3000	features: 0.962

Best score for this clf is 0.966 with 2000 features.

Testing clf:
 (<class 'sklearn.svm._classes.LinearSVC'>, {'max_iter': 10000})
Score with 10	features: 0.8
Score with 100	features: 0.951
Score with 250	features: 0.971
Score with 500	features: 0.969
Score with 750	features: 0.969
Score with 1000	features: 0.971
Score with 1500	features: 0.973
Score with 2000	features: 0.973
Score with 3000	features: 0.978

Best score for this clf is 0.978 with 3000 features.

Testing clf:
 (<class 'sklearn.svm._classes.SVC'>, {'kernel': 'rbf', 'gamma': 0.7})
Score with 10	features: 0.818
Score with 100	features: 0.935
Score with 250	features: 0.955
Sco

## Evaluation

In [ ]:
print ("Score using dev set:\t",str(round(best_score,4)))
test_score = score_svm_classifier(best_svm_clf, X_test, Y_test, best_sent_analysis)
print ("Score using test set:\t",str(round(test_score,4)))
print ("Variance of:",str(round(test_score-best_score,5)))
Y_dev_predictions = predict_svm_classifier(best_svm_clf, X_dev, best_sent_analysis)
print("\nDev set classification report")
print(sklmet.classification_report(Y_dev, Y_dev_predictions))
Y_test_predictions = predict_svm_classifier(best_svm_clf, X_test, best_sent_analysis)
print("\nTest set classification report")
print(sklmet.classification_report(Y_test, Y_test_predictions))
plot_confusion_matrix(best_svm_clf, X_dev, Y_dev, best_sent_analysis,"Dev set confusion matrix" )
plot_confusion_matrix(best_svm_clf, X_test, Y_test, best_sent_analysis,"Test set confusion matrix")


Score using dev set:	 0.9775
Score using test set:	 0.9663
Variance of: -0.01124

Dev set classification report
               precision    recall  f1-score   support

     business       0.98      0.98      0.98        86
entertainment       0.94      0.97      0.95        74
     politics       0.96      1.00      0.98        74
        sport       1.00      0.99      1.00       104
         tech       1.00      0.95      0.98       107

     accuracy                           0.98       445
    macro avg       0.97      0.98      0.98       445
 weighted avg       0.98      0.98      0.98       445


Test set classification report
               precision    recall  f1-score   support

     business       0.95      0.95      0.95       110
entertainment       1.00      0.95      0.97        80
     politics       0.90      0.98      0.93        81
        sport       1.00      0.99      0.99       100
         tech       0.99      0.97      0.98        74

     accuracy             